<a href="https://colab.research.google.com/github/AnnaK8090/CIND-820_Big-Data-Analytics-Project/blob/main/2_Filtering_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Importing libraries:
import numpy as np 
import pandas as pd   

In [41]:
# 2. Loading csv file and saving it into a dataframe:
masterDF = pd.read_csv('MasterDF.csv', on_bad_lines='skip')

In [42]:
masterDF.shape

(113210, 40)

In [44]:
# 3. Since order_ID (~ transaction id) might have more than 1 review we will aggregate dataframe by 
# key 3 fields (customer_unique_id, product_id and category) and then choose max review score: 
   
masterDF_grouped = masterDF.groupby(['customer_unique_id','product_id','product_category_name_english'])['review_score'].max()
masterDF_grouped = masterDF_grouped.reset_index()
masterDF_grouped.shape

(97689, 4)

In [45]:
masterDF_grouped.to_csv ('masterDF_grouped.csv', index = None, header=True)

In [18]:
masterDF_grouped.head()

,customer_unique_id,product_id,product_category_name_english,review_score
0,0000366f3b9a7992bf8c76cfdf3221e2,372645c7439f9661fbbacfd129aa92ec,bed_bath_table,5
1,0000b849f77a49e4a4ce2b2a4ca5be3f,5099f7000472b634fea8304448d20825,health_beauty,4
2,0000f46a3911fa3c0805444483337064,64b488de448a5324c4134ea39c28a34b,stationery,3
3,0000f6ccb0745a6a4b88665a16c9f078,2345a354a6f2033609bbf62bf5be9ef6,telephony,4
4,0004aac84e0df4da2b147fca70cf8255,c72e18b3fe2739b8d24ebf3102450f37,telephony,5


In [51]:
#4. OPTION 1 
# Recommender systems have a problem known as user cold-start, in which is hard do provide personalized recommendations for users with none or a very few number of consumed items, 
# due to the lack of information to model their preferences. For this reason, we are keeping in the dataset only users with at least 2 reviews.

# firstly we count products per customer (in a separate dataframe):
ProductsPerCustomer = masterDF_grouped.groupby(['customer_unique_id'])['product_id'].agg('count').reset_index()

#secondly we filter the dataframe by condition >2:
ProductsGreater2PerCustomer = ProductsPerCustomer.loc[ProductsPerCustomer['product_id'] >=3]

#thirdly we filter the initial dataframe - only those customers that bought >=2 products will remain: 
result1 = masterDF_grouped[(masterDF_grouped.customer_unique_id.isin(ProductsGreater2PerCustomer.customer_unique_id))]
result1.shape

(2491, 4)

In [52]:
# 5. OPTION 1 Descriptive statistics:
# Number of users
print('The dataset has', result1['customer_unique_id'].nunique(), 'unique users')
# Number of products
print('The ratings dataset has', result1['product_id'].nunique(), 'unique products')
# Number of product categories
print('The ratings dataset has', result1['product_category_name_english'].nunique(), 'unique product categories')
# Number of ratings
print('The ratings dataset has', result1['review_score'].nunique(), 'unique ratings:', sorted(result1['review_score'].unique()))

The dataset has 725 unique users
The ratings dataset has 2136 unique products
The ratings dataset has 61 unique product categories
The ratings dataset has 5 unique ratings: [1, 2, 3, 4, 5]


In [54]:
result1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2491 entries, 238 to 97538
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   customer_unique_id             2491 non-null   object
 1   product_id                     2491 non-null   object
 2   product_category_name_english  2491 non-null   object
 3   review_score                   2491 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 97.3+ KB


In [53]:
result1.to_csv ('result_3orMore.csv', index = None, header=True)

In [ ]:
# 6. OPTION 2 To make the calculation manageable by the Google Colab memory
# we will reduce the number of records to only those PRODUCTS that have more than 50 reviews:

# firstly we count reviews per product (in a separate dataframe):
ReviewsPerProduct = masterDF_grouped.groupby('product_id').agg(mean_rating = ('review_score', 'mean'), number_of_reviews = ('review_score', 'count')).reset_index()
# secondly we keep the products with over 50 reviews
ReviewsPerProduct_GT50 = ReviewsPerProduct[ReviewsPerProduct['number_of_reviews']>50]
# thirdly filter the initial dataframe (how='inner' and on='product_id') to ensure that only the products with more than 50 reviews are included.
result2 = pd.merge(masterDF_grouped, ReviewsPerProduct_GT50[['product_id']], on='product_id', how='inner')
result2.shape

(14632, 4)

In [ ]:
# 7. OPTION 2 Descriptive statistics:
# Number of users
print('The dataset has', result2['customer_unique_id'].nunique(), 'unique users')
# Number of products
print('The ratings dataset has', result2['product_id'].nunique(), 'unique products')
# Number of product categories
print('The ratings dataset has', result2['product_category_name_english'].nunique(), 'unique product categories')
# Number of ratings
print('The ratings dataset has', result2['review_score'].nunique(), 'unique ratings:', sorted(result1['review_score'].unique()))

The dataset has 14352 unique users
The ratings dataset has 139 unique products
The ratings dataset has 25 unique product categories
The ratings dataset has 5 unique ratings: [1, 2, 3, 4, 5]


In [ ]:
result2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14632 entries, 0 to 14631
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   customer_unique_id             14632 non-null  object
 1   product_id                     14632 non-null  object
 2   product_category_name_english  14632 non-null  object
 3   review_score                   14632 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 571.6+ KB


In [ ]:
result2.to_csv ('result2.csv', index = None, header=True)